# Total Energy Consumption Equation

The total energy consumption ( $P$ ) can be represented as:

$$
P = \beta_1 \cdot N_{\text{servers}} + \beta_2 \cdot N_{\text{network\_switches\_poe}} + \beta_3 \cdot N_{\text{network\_switches\_non\_poe}} + \beta_4 \cdot N_{\text{hard\_drives}} + \beta_5 \cdot N_{\text{ssds}}
$$

Where:

- $N_{\text{servers}}$ = Number of active servers
- $N_{\text{network\_switches\_poe}}$ = Number of active network switches (PoE) aggregated with non-PoE
- $N_{\text{network\_switches\_non\_poe}}$ = Number of active network switches (Non-PoE) aggregated with PoE
- $N_{\text{hard\_drives}}$ = Number of active hard drives (HDD)
- $N_{\text{ssds}}$ = Number of active solid-state drives (SSD)

And the coefficients are:

- $\beta_1$ = 0.2111872146 kWh per server per hour
- $\beta_2$ is an average of 0.1 kWh to 0.25 kWh per PoE switch per hour
- $\beta_3$ is an average of 0.02 kWh to 0.05 kWh per non-PoE switch per hour
- $\beta_4$ is an average of 0.007 kWh to 0.0105 kWh per HDD per hour
- $\beta_5$ = 0.007 kWh per SSD per hour

### Sources
[Server Source](https://cc-techgroup.com/data-center-energy-consumption/#:~:text=this%20work%20efficiently.-,How%20Many%20kWh%20Does%20a%20Server%20Use%3F,to%201%2C900%20kWh%20every%20year.)

Average annual power may be around 1,800 to 1,900 kWh every year per server (divide by 365 then 24 then 4): 0.2111872146 kWh per hour

[Network Switch Source](https://www.genuinemodules.com/how-much-power-does-a-24-port-switch-use_a8539#:~:text=However%2C%20as%20a%20general%20estimate,requirements%20of%20the%20connected%20devices.)

However, as a general estimate, a non-PoE 24 port switch may consume around 20-50 watts of power, while a PoE-enabled switch may consume around 100-250 watts or more, depending on the power requirements of the connected devices.
PoE: 0.1 kWh - 0.25 kWh per hour
Non-PoE: 0.02 kWh - 0.05 kWh per hour

[HDD & SDD Source](https://dataspan.com/blog/how-much-energy-do-data-centers-use/#:~:text=Depending%20on%20disk%20size%2C%20an,around%206%20watts%20per%20disk.)

Depending on disk size, an HDD can use anywhere from 6 to 9 watts at maximum capacity. While this is lower than in previous years, power requirements for SSDs have remained mostly constant at around 6 watts per disk.

HDD: 0.007 kWh - 0.0105 kWh per hour
SSD: 0.007 kWh per hour

In [1]:
cd ..

c:\Users\Usuario\OneDrive\Documents\IE\3. Trimestre\Venture Lab & Capstone\Capstone\Tech side\vl_optimizer


In [2]:
import joblib

def calculate_energy(server_count, network_switch_count, hdd_count, ssd_count):
    power_consumption_per_hour = {
    'server': 0.2111872146,  # kWh per hour
    # 'network_switch': (0.1748 + 0.035) / 2,  # kWh per hour
    'network_switch_poe': (((100 / 1000 ) + (249.6 / 1000))/2),  # Convert Wh to kWh per hour
    'network_switch_non_poe': (((20 / 1000 ) + (50 / 1000))/2),  # Convert Wh to kWh per hour
    'hdd': (((0.7 / 1000 ) + (1.05 / 1000))/2),  # Convert Wh to kWh per hour
    'ssd': 0.7 / 1000  # Convert Wh to kWh per hour
}
    
    total_energy = (
        server_count * power_consumption_per_hour['server'] +
        network_switch_count * power_consumption_per_hour['network_switch'] +
        hdd_count * power_consumption_per_hour['hdd'] +
        ssd_count * power_consumption_per_hour['ssd']
    )
    
    return total_energy

features = ['server', 'network_switch_poe', 'network_switch_non_poe', 'hdd', 'ssd']
coefficients = {
    'server': 0.2111872146,  # kWh per hour
    # 'network_switch': (0.1748 + 0.035) / 2,  # kWh per hour
    'network_switch_poe': (((100 / 1000 ) + (249.6 / 1000))/2),  # Convert Wh to kWh per hour
    'network_switch_non_poe': (((20 / 1000 ) + (50 / 1000))/2),  # Convert Wh to kWh per hour
    'hdd': (((0.7 / 1000 ) + (1.05 / 1000))/2),  # Convert Wh to kWh per hour
    'ssd': 0.7 / 1000  # Convert Wh to kWh per hour
}

model_params = {
    'coefficients': coefficients,
    'features': features
}

model_filename = 'equation_modeling/models/data_center_consumption_model.joblib'
joblib.dump(model_params, model_filename)



['equation_modeling/models/data_center_consumption_model.joblib']

In [3]:
data_center_model = joblib.load('equation_modeling/models/data_center_consumption_model.joblib')

print(data_center_model)

{'coefficients': {'server': 0.2111872146, 'network_switch_poe': 0.1748, 'network_switch_non_poe': 0.035, 'hdd': 0.0008750000000000001, 'ssd': 0.0007}, 'features': ['server', 'network_switch_poe', 'network_switch_non_poe', 'hdd', 'ssd']}


To create a simple equation for the minimum number of active hardware based on the time of day, we can consider different times and the corresponding minimum requirements for each type of hardware. Let's define some time-based categories and their respective minimum requirements. For simplicity, let's divide the day into three periods: peak hours, off-peak hours, and regular hours.

### Time Periods:
1. **Peak Hours (e.g., 8 AM - 6 PM)**
2. **Regular Hours (e.g., 6 PM - 10 PM)**
3. **Off-Peak Hours (e.g., 10 PM - 8 AM)**

### Assumptions:
- Peak Hours require the highest number of active hardware.
- Regular Hours require a moderate number of active hardware.
- Off-Peak Hours require the lowest number of active hardware.

### Base Minimum Numbers:
These numbers should be adjusted based on your specific operational needs. For illustration:

- **Number of Servers:**
  - Peak Hours: 50
  - Regular Hours: 40
  - Off-Peak Hours: 30

- **Number of Network Switches (PoE):**
  - Peak Hours: 10
  - Regular Hours: 8
  - Off-Peak Hours: 6

- **Number of Network Switches (Non-PoE):**
  - Peak Hours: 5
  - Regular Hours: 4
  - Off-Peak Hours: 3

- **Number of Hard Drives (HDD):**
  - Peak Hours: 100
  - Regular Hours: 80
  - Off-Peak Hours: 60

- **Number of SSDs:**
  - Peak Hours: 50
  - Regular Hours: 40
  - Off-Peak Hours: 30

### Equations:
Let \( t \) be the time of day in hours (0-23).

1. **Number of Active Servers (\( N_s \))**:
\[ 
N_s = 
\begin{cases} 
50 & \text{if } 8 \leq t < 18 \\
40 & \text{if } 18 \leq t < 22 \\
30 & \text{if } 22 \leq t < 8 
\end{cases}
\]

2. **Number of Active Network Switches (PoE) (\( N_{ns\_poe} \))**:
\[ 
N_{ns\_poe} = 
\begin{cases} 
10 & \text{if } 8 \leq t < 18 \\
8 & \text{if } 18 \leq t < 22 \\
6 & \text{if } 22 \leq t < 8 
\end{cases}
\]

3. **Number of Active Network Switches (Non-PoE) (\( N_{ns\_non\_poe} \))**:
\[ 
N_{ns\_non\_poe} = 
\begin{cases} 
5 & \text{if } 8 \leq t < 18 \\
4 & \text{if } 18 \leq t < 22 \\
3 & \text{if } 22 \leq t < 8 
\end{cases}
\]

4. **Number of Active Hard Drives (HDD) (\( N_{hdd} \))**:
\[ 
N_{hdd} = 
\begin{cases} 
100 & \text{if } 8 \leq t < 18 \\
80 & \text{if } 18 \leq t < 22 \\
60 & \text{if } 22 \leq t < 8 
\end{cases}
\]

5. **Number of Active SSDs (\( N_{ssd} \))**:
\[ 
N_{ssd} = 
\begin{cases} 
50 & \text{if } 8 \leq t < 18 \\
40 & \text{if } 18 \leq t < 22 \\
30 & \text{if } 22 \leq t < 8 
\end{cases}
\]

Using these equations, you can adjust the minimum number of active hardware based on the time of day. You can tailor these numbers further based on your specific operational data and requirements.

In [4]:
from pyomo.environ import ConcreteModel, Var, Constraint, NonNegativeIntegers, Objective, minimize

def get_min_active_hardware(time_of_day):
    if 8 <= time_of_day < 18:
        return {
            "servers": 50,
            "network_switches_poe": 10,
            "network_switches_non_poe": 5,
            "hard_drives": 100,
            "ssds": 50
        }
    elif 18 <= time_of_day < 22:
        return {
            "servers": 40,
            "network_switches_poe": 8,
            "network_switches_non_poe": 4,
            "hard_drives": 80,
            "ssds": 40
        }
    else:
        return {
            "servers": 30,
            "network_switches_poe": 6,
            "network_switches_non_poe": 3,
            "hard_drives": 60,
            "ssds": 30
        }

def create_model(time_of_day):
    min_hardware = get_min_active_hardware(time_of_day)
    
    model = ConcreteModel()
    
    # Define variables
    model.servers = Var(within=NonNegativeIntegers)
    model.network_switches_poe = Var(within=NonNegativeIntegers)
    model.network_switches_non_poe = Var(within=NonNegativeIntegers)
    model.hard_drives = Var(within=NonNegativeIntegers)
    model.ssds = Var(within=NonNegativeIntegers)
    
    # Define constraints based on minimum hardware requirements
    model.min_servers = Constraint(expr=model.servers >= min_hardware['servers'])
    model.min_network_switches_poe = Constraint(expr=model.network_switches_poe >= min_hardware['network_switches_poe'])
    model.min_network_switches_non_poe = Constraint(expr=model.network_switches_non_poe >= min_hardware['network_switches_non_poe'])
    model.min_hard_drives = Constraint(expr=model.hard_drives >= min_hardware['hard_drives'])
    model.min_ssds = Constraint(expr=model.ssds >= min_hardware['ssds'])
    
    # Example objective: Minimize total number of components
    model.objective = Objective(expr=model.servers + model.network_switches_poe + model.network_switches_non_poe + model.hard_drives + model.ssds, sense=minimize)
    
    return model

# Example usage:
time_of_day = 10  # Example time of day
model = create_model(time_of_day)

# Solve the model (you need a solver for this part)
from pyomo.opt import SolverFactory
solver = SolverFactory('glpk')  # Ensure you have the solver installed
result = solver.solve(model)

# Display results
model.servers.display()
model.network_switches_poe.display()
model.network_switches_non_poe.display()
model.hard_drives.display()
model.ssds.display()

servers : Size=1, Index=None
    Key  : Lower : Value : Upper : Fixed : Stale : Domain
    None :     0 :  50.0 :  None : False : False : NonNegativeIntegers
network_switches_poe : Size=1, Index=None
    Key  : Lower : Value : Upper : Fixed : Stale : Domain
    None :     0 :  10.0 :  None : False : False : NonNegativeIntegers
network_switches_non_poe : Size=1, Index=None
    Key  : Lower : Value : Upper : Fixed : Stale : Domain
    None :     0 :   5.0 :  None : False : False : NonNegativeIntegers
hard_drives : Size=1, Index=None
    Key  : Lower : Value : Upper : Fixed : Stale : Domain
    None :     0 : 100.0 :  None : False : False : NonNegativeIntegers
ssds : Size=1, Index=None
    Key  : Lower : Value : Upper : Fixed : Stale : Domain
    None :     0 :  50.0 :  None : False : False : NonNegativeIntegers
